In [1]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image, ImageSequence

# Inicializar Mediapipe para detección de landmarks faciales
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Cargar el GIF y convertirlo a una lista de fotogramas
gif = Image.open("rainbow.gif")
gif_frames = [frame.copy() for frame in ImageSequence.Iterator(gif)]

def gif_overlay(img, frame_position, scale_width):
    """Superpone el GIF escalado en la posición especificada de la imagen."""
    frame = gif_frames[int(cv2.getTickCount() % len(gif_frames)) % len(gif_frames)]  # Selección de cuadro de GIF
    frame = frame.resize((scale_width, int(scale_width * frame.size[1] / frame.size[0])), Image.LANCZOS)
    frame = frame.convert("RGBA")
    frame_np = np.array(frame)
    alpha = frame_np[:, :, 3] / 255.0
    frame_np = cv2.cvtColor(frame_np, cv2.COLOR_RGBA2RGB)
    
    # Obtener la posición para el GIF
    x, y = frame_position
    x = max(0, x - frame_np.shape[1] // 2)
    y = max(0, y - frame_np.shape[0] // 2)
    
    # Superponer el GIF en la imagen
    for c in range(0, 3):
        img[y:y+frame_np.shape[0], x:x+frame_np.shape[1], c] = (alpha * frame_np[:, :, c] +
                                                                 (1 - alpha) * img[y:y+frame_np.shape[0], x:x+frame_np.shape[1], c])

def mouth_open(landmarks, threshold=0.05):
    """Calcula si la boca está abierta basado en la distancia entre puntos específicos."""
    top_lip = landmarks[13]  # Landmark del labio superior
    bottom_lip = landmarks[14]  # Landmark del labio inferior
    mouth_height = abs(bottom_lip.y - top_lip.y)
    return mouth_height > threshold

# Inicializar captura de video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Obtener coordenadas de los landmarks de la boca
            mouth_top = face_landmarks.landmark[13]  # Labio superior
            mouth_bottom = face_landmarks.landmark[14]  # Labio inferior
            mouth_left = face_landmarks.landmark[78]  # Lado izquierdo de la boca
            mouth_right = face_landmarks.landmark[308]  # Lado derecho de la boca
            
            # Calcular posición y ancho de la boca
            mouth_center_x = int((mouth_top.x + mouth_bottom.x) / 2 * frame.shape[1])
            mouth_center_y = int((mouth_top.y + mouth_bottom.y) / 2 * frame.shape[0])
            mouth_width = int(abs(mouth_right.x - mouth_left.x) * frame.shape[1])

            # Verificar si la boca está abierta
            if mouth_open(face_landmarks.landmark):
                gif_overlay(frame, (mouth_center_x, mouth_center_y), mouth_width)

    cv2.imshow("Mouth Open Detection with GIF", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
